In [2]:
"""
Dedalus script simulating 2D horizontally-periodic Rayleigh-Benard convection.
This script demonstrates solving a 2D Cartesian initial value problem. It can
be ran serially or in parallel, and uses the built-in analysis framework to save
data snapshots to HDF5 files. The `plot_snapshots.py` script can be used to
produce plots from the saved data. It should take about 5 cpu-minutes to run.

The problem is non-dimensionalized using the box height and freefall time, so
the resulting thermal diffusivity and viscosity are related to the Prandtl
and Rayleigh numbers as:
    kappa = (Rayleigh * Prandtl)**(-1/2)
    nu = (Rayleigh / Prandtl)**(-1/2)

For incompressible hydro with two boundaries, we need two tau terms for each the
velocity and buoyancy. Here we choose to use a first-order formulation, putting
one tau term each on auxiliary first-order gradient variables and the others in
the PDE, and lifting them all to the first derivative basis. This formulation puts
a tau term in the divergence constraint, as required for this geometry.

To run and plot using e.g. 4 processes:
    $ mpiexec -n 4 python3 rayleigh_benard.py
    $ mpiexec -n 4 python3 plot_snapshots.py snapshots/*.h5
"""


'\nDedalus script simulating 2D horizontally-periodic Rayleigh-Benard convection.\nThis script demonstrates solving a 2D Cartesian initial value problem. It can\nbe ran serially or in parallel, and uses the built-in analysis framework to save\ndata snapshots to HDF5 files. The `plot_snapshots.py` script can be used to\nproduce plots from the saved data. It should take about 5 cpu-minutes to run.\n\nThe problem is non-dimensionalized using the box height and freefall time, so\nthe resulting thermal diffusivity and viscosity are related to the Prandtl\nand Rayleigh numbers as:\n    kappa = (Rayleigh * Prandtl)**(-1/2)\n    nu = (Rayleigh / Prandtl)**(-1/2)\n\nFor incompressible hydro with two boundaries, we need two tau terms for each the\nvelocity and buoyancy. Here we choose to use a first-order formulation, putting\none tau term each on auxiliary first-order gradient variables and the others in\nthe PDE, and lifting them all to the first derivative basis. This formulation puts\na tau 

In [3]:
import numpy as np
import dedalus.public as d3
import logging
logger = logging.getLogger(__name__)

In [4]:
# Parameters
Lx,Ly, Lz = 4,4,1
Nx, Nz = 256, 64
Rayleigh = 2e6
Prandtl = 1
dealias = 3/2
stop_sim_time = 50
timestepper = d3.RK222
max_timestep = 0.125
dtype = np.float64

In [5]:
# Bases
coords = d3.CartesianCoordinates('x','y', 'z')
dist = d3.Distributor(coords, dtype=dtype)
xbasis = d3.RealFourier(coords['x'], size=Nx, bounds=(0, Lx), dealias=dealias)
ybasis= d3.RealFourier(coords['y'], size=Nx, bounds=(0, Ly), dealias=dealias)
zbasis = d3.ChebyshevT(coords['z'], size=Nz, bounds=(0, Lz), dealias=dealias)

In [6]:
# Fields
p = dist.Field(name='p', bases=(xbasis,ybasis,zbasis))
b = dist.Field(name='b', bases=(xbasis,ybasis,zbasis))
u = dist.VectorField(coords, name='u', bases=(xbasis,ybasis,zbasis))
tau_p = dist.Field(name='tau_p')
tau_b1 = dist.Field(name='tau_b1', bases=(xbasis,ybasis))
tau_b2 = dist.Field(name='tau_b2', bases=(xbasis,ybasis))
tau_u1 = dist.VectorField(coords, name='tau_u1', bases=(xbasis,ybasis))
tau_u2 = dist.VectorField(coords, name='tau_u2', bases=(xbasis,ybasis))
# Substitutions
kappa = (Rayleigh * Prandtl)**(-1/2)
nu = (Rayleigh / Prandtl)**(-1/2)
x,y,z = dist.local_grids(xbasis,ybasis,zbasis)
ex,ey, ez = coords.unit_vector_fields(dist)
lift_basis = zbasis.derivative_basis(1)
lift = lambda A: d3.Lift(A, lift_basis, -1)
grad_u = d3.grad(u) + ez*lift(tau_u1) # First-order reduction
grad_b = d3.grad(b) + ez*lift(tau_b1) # First-order reduction


In [7]:
# Problem
# First-order form: "div(f)" becomes "trace(grad_f)"
# First-order form: "lap(f)" becomes "div(grad_f)"
problem = d3.IVP([p, b, u, tau_p, tau_b1, tau_b2, tau_u1, tau_u2], namespace=locals())
problem.add_equation("trace(grad_u) + tau_p= 0")
problem.add_equation("dt(b) - kappa*div(grad_b) + lift(tau_b2) = - u@grad(b)")
problem.add_equation("dt(u) - nu*div(grad_u) + grad(p) - b*ez + lift(tau_u2) = - u@grad(u)")
problem.add_equation("b(z=0) = Lz")
problem.add_equation("u(z=0) = 0")
problem.add_equation("b(z=Lz) = 0")
problem.add_equation("u(z=Lz)= 0")
problem.add_equation("integ(p) = 0") # Pressure gauge

{'LHS': Integrate(Integrate(Integrate(<Field 22623195880688>))),
 'RHS': 0,
 'condition': 'True',
 'tensorsig': (),
 'dtype': numpy.float64,
 'M': 0,
 'L': Integrate(Integrate(Integrate(<Field 22623195880688>))),
 'F': <Field 22623195675760>,
 'domain': <dedalus.core.domain.Domain at 0x14935f8dc310>,
 'matrix_dependence': array([ True,  True,  True]),
 'matrix_coupling': array([False, False,  True])}

In [8]:
# Solver
solver = problem.build_solver(timestepper)
solver.stop_sim_time = stop_sim_time


2023-06-09 18:22:53,794 subsystems 0/1 INFO :: Building subproblem matrices 1/16384 (~0%) Elapsed: 2s, Remaining: 9h 30m 03s, Rate: 4.8e-01/s
2023-06-09 18:23:01,723 subsystems 0/1 INFO :: Building subproblem matrices 15/16384 (~0%) Elapsed: 10s, Remaining: 3h 02m 11s, Rate: 1.5e+00/s
2023-06-09 18:23:11,966 subsystems 0/1 INFO :: Building subproblem matrices 33/16384 (~0%) Elapsed: 20s, Remaining: 2h 47m 18s, Rate: 1.6e+00/s
2023-06-09 18:23:22,079 subsystems 0/1 INFO :: Building subproblem matrices 51/16384 (~0%) Elapsed: 30s, Remaining: 2h 42m 07s, Rate: 1.7e+00/s
2023-06-09 18:23:32,232 subsystems 0/1 INFO :: Building subproblem matrices 69/16384 (~0%) Elapsed: 41s, Remaining: 2h 39m 42s, Rate: 1.7e+00/s
2023-06-09 18:23:41,831 subsystems 0/1 INFO :: Building subproblem matrices 86/16384 (~1%) Elapsed: 50s, Remaining: 2h 38m 19s, Rate: 1.7e+00/s


KeyboardInterrupt: 

In [ ]:
# Initial conditio
b.fill_random('g', seed=42, distribution='normal', scale=1e-3) # Random noise
b['g'] *= z * (Lz - z) # Damp noise at walls
b['g'] += Lz - z # Add linear background

In [ ]:
# Analysis
snapshots = solver.evaluator.add_file_handler('snapshots', sim_dt=0.25, max_writes=50)
snapshots.add_task(d3.Average(u,'y'), name='average buoyancy')

In [ ]:
# CFL
CFL = d3.CFL(solver, initial_dt=max_timestep, cadence=10, safety=0.5, threshold=0.05,
             max_change=1.5, min_change=0.5, max_dt=max_timestep)
CFL.add_velocity(u)

In [ ]:
# Flow properties
flow = d3.GlobalFlowProperty(solver, cadence=10)
flow.add_property(np.sqrt(u@u)/nu, name='Re')


In [ ]:
# Main loop
startup_iter = 10
try:
    logger.info('Starting main loop')
    while solver.proceed:
        timestep = CFL.compute_timestep()
        solver.step(timestep)
        if (solver.iteration-1) % 10 == 0:
            max_Re = flow.max('Re')
            logger.info('Iteration=%i, Time=%e, dt=%e, max(Re)=%f' %(solver.iteration, solver.sim_time, timestep, max_Re))
except:
    logger.error('Exception raised, triggering end of main loop.')
    raise
finally:
    solver.log_stats()